In [8]:
from bs4 import BeautifulSoup
import requests
import os

In [ ]:
# leer WikiURLs.txt para crear lista de URLs
# TODO modificar para evitar errores
wikipedia_links = []
file_path = "WikiURLs.txt"
with open(file_path, "r", encoding='utf-8') as file:
    for line in file:
        line = line.strip()
        if "wikipedia.org" in line:
            wikipedia_links.append(line)

#print(wikipedia_links)

['https://en.wikipedia.org/wiki/Art', 'https://en.wikipedia.org/wiki/Music', 'https://en.wikipedia.org/wiki/Literature', 'https://en.wikipedia.org/wiki/Dance', 'https://en.wikipedia.org/wiki/Film', 'https://en.wikipedia.org/wiki/Theatre', 'https://en.wikipedia.org/wiki/Cuisine', 'https://en.wikipedia.org/wiki/Fashion', 'https://en.wikipedia.org/wiki/Architecture', 'https://en.wikipedia.org/wiki/Photography', 'https://en.wikipedia.org/wiki/Earth', 'https://en.wikipedia.org/wiki/Continent', 'https://en.wikipedia.org/wiki/Country', 'https://en.wikipedia.org/wiki/Mountain', 'https://en.wikipedia.org/wiki/River', 'https://en.wikipedia.org/wiki/Desert', 'https://en.wikipedia.org/wiki/Ocean', 'https://en.wikipedia.org/wiki/Climate', 'https://en.wikipedia.org/wiki/Ecosystem', 'https://en.wikipedia.org/wiki/Map', 'https://en.wikipedia.org/wiki/Medicine', 'https://en.wikipedia.org/wiki/Nutrition', 'https://en.wikipedia.org/wiki/Physical_exercise', 'https://en.wikipedia.org/wiki/Mental_health', '

In [ ]:
for link in wikipedia_links:
    res = requests.get(link)
    soup = BeautifulSoup(res.text, 'html.parser')

    #print(soup.prettify())
    output_html = "HTMLResults/" + link.split("/wiki/")[-1].replace("_", " ") + ".txt"
    print(output_html)
    with open(output_html, "w", encoding='utf-8') as file:
        file.write(soup.prettify())

    print(f"Data saved to {output_html}")

HTMLResults/Art.txt
Data saved to HTMLResults/Art.txt
HTMLResults/Music.txt
Data saved to HTMLResults/Music.txt
HTMLResults/Literature.txt
Data saved to HTMLResults/Literature.txt
HTMLResults/Dance.txt
Data saved to HTMLResults/Dance.txt
HTMLResults/Film.txt
Data saved to HTMLResults/Film.txt
HTMLResults/Theatre.txt
Data saved to HTMLResults/Theatre.txt
HTMLResults/Cuisine.txt
Data saved to HTMLResults/Cuisine.txt
HTMLResults/Fashion.txt
Data saved to HTMLResults/Fashion.txt
HTMLResults/Architecture.txt
Data saved to HTMLResults/Architecture.txt
HTMLResults/Photography.txt
Data saved to HTMLResults/Photography.txt
HTMLResults/Earth.txt
Data saved to HTMLResults/Earth.txt
HTMLResults/Continent.txt
Data saved to HTMLResults/Continent.txt
HTMLResults/Country.txt
Data saved to HTMLResults/Country.txt
HTMLResults/Mountain.txt
Data saved to HTMLResults/Mountain.txt
HTMLResults/River.txt
Data saved to HTMLResults/River.txt
HTMLResults/Desert.txt
Data saved to HTMLResults/Desert.txt
HTMLResult

In [9]:
input_dir = "HTMLResults"
output_dir = "TextExtractionResults"

for filename in os.listdir(input_dir):
    input_file_path = os.path.join(input_dir, filename)
    with open(input_file_path, "r", encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')

    headings = [heading.get_text(strip=True) for heading in soup.find_all(["h1", "h2", "h3", "h4", "h5", "h6"])]
    paragraphs = [p.get_text(strip=True) for p in soup.find_all("p")]

    output_content = "\n".join(headings) + "\n\n" + "\n".join(paragraphs)
    output_file_path = os.path.join(output_dir, filename)
    with open(output_file_path, "w", encoding='utf-8') as file:
        file.write(output_content)
